In [7]:
import numpy as np
import polars as pd
import torch
from torch import nn, functional as F

In [ ]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import os

urlretrieve("https://files.grouplens.org/datasets/movielens/ml-32m.zip", "../data/movielens.zip")
ZipFile("../data/movielens.zip", "r").extractall('../data/')
os.remove('../data/movielens.zip')


In [5]:
urlretrieve("https://www.kaggle.com/api/v1/datasets/download/asaniczka/tmdb-movies-dataset-2023-930k-movies", "../data/tmdb.zip")
ZipFile("../data/tmdb.zip", "r").extractall('../data/tmdb_dataset/')
os.remove('../data/tmdb.zip')

In [8]:
movies_df = pd.read_csv('../data/tmdb_dataset/TMDB_movie_dataset_v11.csv')
ratings_df = pd.read_csv('../data/ml-32m/ratings.csv')
links_df = pd.read_csv('../data/ml-32m/links.csv')

In [9]:
movies_df = links_df.join(movies_df,left_on="tmdbId",right_on="id")
movies_df = movies_df.drop(["title","status","backdrop_path","homepage","imdb_id","imdbId","poster_path"])
movies_df.head()

movieId,tmdbId,vote_average,vote_count,release_date,revenue,runtime,adult,budget,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,keywords
i64,i64,f64,i64,str,i64,i64,bool,i64,str,str,str,f64,str,str,str,str,str,str
79132,27205,8.364,34495,"""2010-07-15""",825532764,148,false,160000000,"""en""","""Inception""","""Cobb, a skilled thief who comm…",83.952,"""Your mind is the scene of the …","""Action, Science Fiction, Adven…","""Legendary Pictures, Syncopy, W…","""United Kingdom, United States …","""English, French, Japanese, Swa…","""rescue, mission, dream, airpla…"
109487,157336,8.417,32571,"""2014-11-05""",701729206,169,false,165000000,"""en""","""Interstellar""","""The adventures of a group of e…",140.241,"""Mankind was born on Earth. It …","""Adventure, Drama, Science Fict…","""Legendary Pictures, Syncopy, L…","""United Kingdom, United States …","""English""","""rescue, future, spacecraft, ra…"
58559,155,8.512,30619,"""2008-07-16""",1004558444,152,false,185000000,"""en""","""The Dark Knight""","""Batman raises the stakes in hi…",130.643,"""Welcome to a world without rul…","""Drama, Action, Crime, Thriller""","""DC Comics, Legendary Pictures,…","""United Kingdom, United States …","""English, Mandarin""","""joker, sadism, chaos, secret i…"
72998,19995,7.573,29815,"""2009-12-15""",2923706026,162,false,237000000,"""en""","""Avatar""","""In the 22nd century, a paraple…",79.932,"""Enter the world of Pandora.""","""Action, Adventure, Fantasy, Sc…","""Dune Entertainment, Lightstorm…","""United States of America, Unit…","""English, Spanish""","""future, society, culture clash…"
89745,24428,7.71,29166,"""2012-04-25""",1518815515,143,false,220000000,"""en""","""The Avengers""","""When an unexpected enemy emerg…",98.082,"""Some assembly required.""","""Science Fiction, Action, Adven…","""Marvel Studios""","""United States of America""","""English, Hindi, Russian""","""new york city, superhero, shie…"


In [10]:
genres = movies_df['genres'].map_elements(lambda x: x.split(','),return_dtype=pd.List(pd.String)).explode().unique().to_list()
print("Number of genres: ",len(genres))
production_companies = movies_df['production_companies'].map_elements(lambda x: x.split(','),return_dtype=pd.List(pd.String)).explode().unique().to_list()
print("Number of production_companies: ",len(production_companies))
production_countries = movies_df['production_countries'].map_elements(lambda x: x.split(','),return_dtype=pd.List(pd.String)).explode().unique().to_list()
print("Number of production_countries: ",len(production_countries))
spoken_languages = movies_df['spoken_languages'].map_elements(lambda x: x.split(','),return_dtype=pd.List(pd.String)).explode().unique().to_list()
print("Number of spoken_languages: ",len(spoken_languages))
keywords = movies_df['keywords'].map_elements(lambda x: x.split(','),return_dtype=pd.List(pd.String)).explode().unique().to_list()
print("Number of keywords: ",len(keywords))


Number of genres:  39
Number of production_companies:  56117
Number of production_countries:  358
Number of spoken_languages:  287
Number of keywords:  33602


39